# Rare Earth Elements Patent Co-occurrence Analysis\n## Enhanced with Claude Code AI Capabilities\n\n**Original Analysis**: Riccardo Priore, Centro Patlib – Area Science Park, Trieste\n\n**AI Enhancement Demo**: Live Claude Code demonstration\n\n---\n\n## Background\nThis notebook analyzes **Rare Earth Elements (REE)** patents using the progression:\n1. **Espacenet Search** → Complex query for REE + recycling patents\n2. **PATSTAT Analysis** → Patent families, IPC co-occurrence, citations\n3. **TIP Enhancement** → Advanced analytics and visualization\n4. **🚀 Claude Code AI** → Market correlation, predictive insights, automated reports\n\n### Original Espacenet Search Strategy:\n```\n(((ctxt=(\"rare \" prox/distance<3 \"earth\") AND ctxt=(\"earth\" prox/distance<3 \"element\")) \nOR ctxt=(\"rare \" prox/distance<3 \"metal\") OR ctxt=(\"rare \" prox/distance<3 \"oxide\") \nOR ctxt=(\"light \" prox/distance<3 \"REE\") OR ctxt=(\"heavy \" prox/distance<3 \"REE\")) \nOR ctxt any \"REE\" OR ctxt any \"lanthan*\") AND (ctxt any \"recov*\" OR ctxt any \"recycl*\")\n```\n\n### Key Results from PATSTAT Analysis:\n- **84,905** distinct patent families (keyword-based)\n- **567,012** families (classification-based)\n- **51,315** IPC co-occurrence patterns (2010-2022)\n- Geographic citation analysis across countries

## 1. Setup and Data Loading\n*Claude Enhancement Target: Add market data integration and advanced error handling*

In [ ]:
# Import required libraries\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom datetime import datetime\nimport warnings\nimport os\nwarnings.filterwarnings('ignore')\n\n# Set display options for better output\npd.set_option('display.max_columns', None)\npd.set_option('display.width', None)\npd.set_option('display.max_colwidth', None)\n\nprint(\"Libraries imported successfully!\")\nprint(f\"Analysis started at: {datetime.now()}\")\n\n# PATSTAT imports with comprehensive error handling\nPATSTAT_AVAILABLE = False\nPATSTAT_CONNECTED = False\n\ntry:\n    from epo.tipdata.patstat import PatstatClient\n    from epo.tipdata.patstat.database.models import (\n        TLS201_APPLN, TLS202_APPLN_TITLE, TLS203_APPLN_ABSTR, \n        TLS209_APPLN_IPC, TLS224_APPLN_CPC, TLS212_CITATION\n    )\n    from sqlalchemy import func, and_, or_\n    from sqlalchemy.orm import sessionmaker, aliased\n    \n    PATSTAT_AVAILABLE = True\n    print(\"✅ PATSTAT libraries imported successfully\")\n    \n    # Initialize PATSTAT client with PROD environment (proven working)\n    environment = 'PROD'  # PROVEN WORKING in our tests\n    \n    print(f\"Connecting to PATSTAT {environment} environment...\")\n    patstat = PatstatClient(env=environment)\n    db = patstat.orm()\n    \n    print(f\"✅ Connected to PATSTAT {environment} environment\")\n    print(f\"Database engine: {db.bind}\")\n    \n    # Test table access\n    try:\n        test_result = db.query(TLS201_APPLN.docdb_family_id).limit(1).first()\n        PATSTAT_CONNECTED = True\n        print(\"✅ Table access test successful\")\n    except Exception as table_error:\n        print(f\"❌ Table access failed: {table_error}\")\n        print(\"⚠️  Issue: BigQuery cannot locate PATSTAT tables in the current configuration\")\n        print(\"🔄 Will use enhanced demo data that replicates real PATSTAT patterns\")\n        PATSTAT_CONNECTED = False\n    \nexcept Exception as e:\n    print(f\"❌ PATSTAT setup failed: {e}\")\n    print(\"🔄 Running in demo mode with realistic REE patent data\")\n    PATSTAT_AVAILABLE = False\n    PATSTAT_CONNECTED = False\n\n# Analysis status summary\nprint(\"\\n📊 Analysis Environment Status:\")\nprint(f\"   PATSTAT Libraries: {'✅ Available' if PATSTAT_AVAILABLE else '❌ Not Available'}\")\nprint(f\"   PATSTAT Connection: {'✅ Connected' if PATSTAT_CONNECTED else '❌ Table Access Issues'}\")\nprint(f\"   Analysis Mode: {'Real Database' if PATSTAT_CONNECTED else 'Enhanced Demo Data'}\")\n\nprint(\"\\n🚀 Ready for Claude Code AI enhancement!\")\nprint(f\"Demo time: {datetime.now()}\")

## 2. REE Patent Search Implementation\n*Enhancement Target: Add real-time Espacenet API integration*

In [ ]:
# REE Patent Search with Robust PATSTAT Integration\n# =================================================\n\n# Riccardo's comprehensive search strategy\nree_keywords = [\n    \"rare earth element\", \"light REE\", \"heavy REE\", \"rare earth metal\",\n    \"rare earth oxide\", \"lanthan\", \"rare earth\", \"neodymium\", \"dysprosium\",\n    \"terbium\", \"europium\", \"yttrium\", \"cerium\", \"lanthanum\", \"praseodymium\"\n]\n\nrecovery_keywords = [\"recov\", \"recycl\", \"extract\", \"separat\", \"purif\"]\n\n# IPC/CPC classification codes from Riccardo's analysis\nkey_classification_codes = [\n    'C22B  19/28', 'C22B  19/30', 'C22B  25/06',  # REE extraction\n    'C04B  18/04', 'C04B  18/06', 'C04B  18/08',  # REE ceramics/materials  \n    'H01M   6/52', 'H01M  10/54',  # REE batteries\n    'C09K  11/01',  # REE phosphors\n    'H01J   9/52',  # REE displays\n    'Y02W30/52', 'Y02W30/56', 'Y02W30/84',  # Recycling technologies\n]\n\ndef execute_ree_patent_search():\n    \"\"\"Execute REE patent search using best available method\"\"\"\n    if PATSTAT_CONNECTED:\n        return execute_real_patstat_search()\n    else:\n        return execute_enhanced_demo_search()\n\ndef execute_real_patstat_search():\n    \"\"\"Real PATSTAT search using proven working patterns - FULL SCALE\"\"\"\n    try:\n        print(\"🔍 Executing Real PATSTAT REE Patent Search - FULL DATASET...\")\n        \n        # Use focused keywords that are proven to work\n        focused_ree_keywords = [\"rare earth\", \"lanthan\", \"neodymium\"]\n        focused_recovery_keywords = [\"recov\", \"recycl\"]\n        \n        print(\"📝 Step 1: Abstract keyword search...\")\n        subquery_abstracts = (\n            db.query(TLS201_APPLN.docdb_family_id, TLS201_APPLN.appln_id, \n                     TLS201_APPLN.appln_filing_date, TLS201_APPLN.appln_nr)\n            .join(TLS203_APPLN_ABSTR, TLS203_APPLN_ABSTR.appln_id == TLS201_APPLN.appln_id)\n            .filter(\n                and_(\n                    TLS201_APPLN.appln_filing_date >= '2010-01-01',\n                    TLS201_APPLN.appln_filing_date <= '2024-12-31',\n                    or_(*[TLS203_APPLN_ABSTR.appln_abstract.contains(kw) for kw in focused_ree_keywords]),\n                    or_(*[TLS203_APPLN_ABSTR.appln_abstract.contains(rw) for rw in focused_recovery_keywords])\n                )\n            ).distinct()\n        )\n        \n        keywords_results = subquery_abstracts.all()\n        keywords_families = [row.docdb_family_id for row in keywords_results]\n        \n        print(f\"✅ Keywords search: {len(keywords_results):,} applications found\")\n        print(f\"   Unique families: {len(set(keywords_families)):,}\")\n        \n        # Step 2: Title search\n        print(\"📝 Step 2: Title keyword search...\")\n        subquery_titles = (\n            db.query(TLS201_APPLN.docdb_family_id, TLS201_APPLN.appln_id,\n                     TLS201_APPLN.appln_filing_date, TLS201_APPLN.appln_nr)\n            .join(TLS202_APPLN_TITLE, TLS202_APPLN_TITLE.appln_id == TLS201_APPLN.appln_id)\n            .filter(\n                and_(\n                    TLS201_APPLN.appln_filing_date >= '2010-01-01',\n                    TLS201_APPLN.appln_filing_date <= '2024-12-31',\n                    or_(*[TLS202_APPLN_TITLE.appln_title.contains(kw) for kw in focused_ree_keywords]),\n                    or_(*[TLS202_APPLN_TITLE.appln_title.contains(rw) for rw in focused_recovery_keywords])\n                )\n            ).distinct()\n        )\n        \n        title_results = subquery_titles.all()\n        title_families = [row.docdb_family_id for row in title_results]\n        \n        print(f\"✅ Title search: {len(title_results):,} applications found\")\n        print(f\"   Unique families: {len(set(title_families)):,}\")\n        \n        # Step 3: Classification search\n        print(\"📝 Step 3: Classification search...\")\n        focused_classification_codes = ['C22B  19/28', 'C22B  19/30', 'C04B  18/04', 'H01M   6/52']\n        \n        subquery_ipc = (\n            db.query(TLS201_APPLN.docdb_family_id, TLS201_APPLN.appln_id,\n                     TLS201_APPLN.appln_filing_date, TLS209_APPLN_IPC.ipc_class_symbol)\n            .join(TLS209_APPLN_IPC, TLS209_APPLN_IPC.appln_id == TLS201_APPLN.appln_id)\n            .filter(\n                and_(\n                    TLS201_APPLN.appln_filing_date >= '2010-01-01',\n                    TLS201_APPLN.appln_filing_date <= '2024-12-31',\n                    func.substr(TLS209_APPLN_IPC.ipc_class_symbol, 1, 11).in_(focused_classification_codes)\n                )\n            ).distinct()\n        )\n        \n        classification_results = subquery_ipc.all()\n        classification_families = [row.docdb_family_id for row in classification_results]\n        \n        print(f\"✅ Classification search: {len(classification_results):,} applications found\")\n        print(f\"   Unique families: {len(set(classification_families)):,}\")\n        \n        # Combine results\n        all_keyword_families = list(set(keywords_families + title_families))\n        all_families = list(set(all_keyword_families + classification_families))\n        intersection_families = list(set(all_keyword_families) & set(classification_families))\n        \n        print(f\"\\n📊 FULL SCALE RESULTS:\")\n        print(f\"   Keywords families: {len(all_keyword_families):,}\")\n        print(f\"   Classification families: {len(set(classification_families)):,}\")\n        print(f\"   Total unique families: {len(all_families):,}\")\n        print(f\"   🎯 High-quality intersection: {len(intersection_families):,}\")\n        \n        # Build final dataset\n        if len(all_families) > 0:\n            print(f\"\\n📝 Building final dataset from {len(all_families):,} families...\")\n            final_query = (\n                db.query(TLS201_APPLN.appln_id, TLS201_APPLN.appln_nr, \n                         TLS201_APPLN.appln_filing_date, TLS201_APPLN.docdb_family_id,\n                         TLS201_APPLN.earliest_filing_year)\n                .filter(TLS201_APPLN.docdb_family_id.in_(all_families))\n                .distinct()\n            )\n            \n            final_results = final_query.all()\n            df_result = pd.DataFrame(final_results, columns=[\n                'appln_id', 'appln_nr', 'appln_filing_date', 'docdb_family_id', 'earliest_filing_year'\n            ])\n            \n            # Add quality indicators\n            df_result['search_method'] = 'Real PATSTAT (Full Keywords + Classification)'\n            df_result['quality_score'] = df_result['docdb_family_id'].apply(\n                lambda x: 1.0 if x in intersection_families else \n                         0.9 if x in all_keyword_families else 0.8\n            )\n            df_result['filing_year'] = pd.to_datetime(df_result['appln_filing_date']).dt.year\n            \n            print(\"✅ Real PATSTAT search successful!\")\n            print(f\"📈 Found {len(df_result):,} REE patent applications\")\n            print(f\"📊 Covering {df_result['docdb_family_id'].nunique():,} unique families\")\n            print(f\"🏆 Average quality score: {df_result['quality_score'].mean():.2f}\")\n            print(f\"📅 Date range: {df_result['filing_year'].min()}-{df_result['filing_year'].max()}\")\n            \n            return df_result\n        else:\n            print(\"⚠️ No results found - switching to demo data\")\n            return execute_enhanced_demo_search()\n        \n    except Exception as e:\n        print(f\"❌ Real PATSTAT search failed: {e}\")\n        print(f\"   Error type: {type(e).__name__}\")\n        print(\"🔄 Falling back to enhanced demo data...\")\n        return execute_enhanced_demo_search()\n\ndef execute_enhanced_demo_search():\n    \"\"\"Enhanced demo search based on Riccardo's actual findings\"\"\"\n    print(\"📊 Executing Enhanced Demo REE Patent Search...\")\n    print(\"🎯 Based on Riccardo's verified PATSTAT analysis results\")\n    \n    # Riccardo's verified results\n    print(\"📈 Riccardo's Original Results:\")\n    print(\"   • 84,905 families (keyword-based)\")\n    print(\"   • 567,012 families (classification-based)\") \n    print(\"   • ~51,315 IPC co-occurrence patterns\")\n    \n    # Create realistic demo dataset\n    np.random.seed(42)\n    n_demo_families = 85\n    \n    countries = ['CN', 'US', 'JP', 'DE', 'KR', 'CA', 'AU', 'FR', 'GB', 'NL']\n    country_weights = [0.35, 0.20, 0.15, 0.08, 0.06, 0.05, 0.04, 0.03, 0.02, 0.02]\n    \n    tech_areas = ['Metallurgy & Extraction', 'Recycling & Recovery', 'Electronics & Magnetics',\n                  'Ceramics & Materials', 'Processing & Separation', 'Other Applications']\n    tech_weights = [0.25, 0.20, 0.18, 0.15, 0.12, 0.10]\n    \n    demo_data = {\n        'appln_id': range(1000000, 1000000 + n_demo_families),\n        'appln_nr': [f'{np.random.choice([\"EP\", \"US\", \"CN\", \"JP\"])}{2010 + i//10}{str(i%10000).zfill(6)}' \n                     for i in range(n_demo_families)],\n        'docdb_family_id': range(500000, 500000 + n_demo_families),\n        'appln_filing_date': pd.date_range('2010-01-01', '2022-12-31', periods=n_demo_families),\n        'geographic_origin': np.random.choice(countries, n_demo_families, p=country_weights),\n        'technology_area': np.random.choice(tech_areas, n_demo_families, p=tech_weights),\n        'search_method': 'Enhanced Demo (Riccardo-based)',\n        'quality_score': np.random.uniform(0.85, 1.0, n_demo_families),\n        'market_relevance': np.random.uniform(0.7, 1.0, n_demo_families)\n    }\n    \n    df_demo = pd.DataFrame(demo_data)\n    df_demo['filing_year'] = pd.to_datetime(df_demo['appln_filing_date']).dt.year\n    df_demo['earliest_filing_year'] = df_demo['filing_year']\n    \n    print(\"✅ Enhanced demo dataset created\")\n    print(f\"📊 Demo families: {len(df_demo):,} (represents ~{len(df_demo)*1000:,} real families)\")\n    \n    return df_demo\n\n# Execute the search\nprint(\"🚀 Starting REE Patent Search - FULL SCALE\")\nprint(\"=\" * 50)\n\nhigh_quality_ree = execute_ree_patent_search()\n\nprint(\"\\n✅ REE Patent Search Complete\")\nprint(f\"📊 Dataset: {len(high_quality_ree):,} patent applications\")\nif len(high_quality_ree) > 0:\n    unique_families = high_quality_ree['docdb_family_id'].nunique() if 'docdb_family_id' in high_quality_ree.columns else 'N/A'\n    print(f\"👨‍👩‍👧‍👦 Unique families: {unique_families:,}\")\nprint(f\"🎯 Search method: {high_quality_ree['search_method'].iloc[0] if len(high_quality_ree) > 0 else 'None'}\")\nif 'quality_score' in high_quality_ree.columns:\n    print(f\"🏆 Average quality score: {high_quality_ree['quality_score'].mean():.2f}\")\n\nprint(\"\\n🚀 Ready for co-occurrence analysis and Claude Code AI enhancement\")\nprint(f\"📈 Expected scale: Similar to Riccardo's 84,905+ families!\")

## 3. Market Data Integration Point\n*🚀 Claude Enhancement: Correlate patents with JRC market data*

In [ ]:
# Market Data Integration Opportunity\nprint(\"📊 JRC Market Data Integration Opportunity:\")\nprint(\"   Available: Rare_Earth_Metals_Market.pdf → Excel data\")\nprint(\"   Available: Rare_Earth_Metals_Recycling_Market.pdf → Excel data\")\nprint(\"\")\nprint(\"🎯 Claude Enhancement Goals:\")\nprint(\"   • Correlate patent filing trends with market prices\")\nprint(\"   • Identify patent-market timing patterns\")\nprint(\"   • Predict technology adoption based on market signals\")\nprint(\"   • Map supply disruptions to innovation responses\")\n\n# Sample market indicators\nmarket_events = {\n    2010: \"REE Crisis Begins\",\n    2011: \"Price Peak (Neodymium $500/kg)\", \n    2014: \"Market Stabilization\",\n    2017: \"EV Market Acceleration\",\n    2019: \"Trade War Impact\",\n    2020: \"COVID Supply Disruption\",\n    2022: \"Green Deal Implementation\"\n}\n\nprint(\"\\n🗓️  Key Market Events for Patent Correlation:\")\nfor year, event in market_events.items():\n    print(f\"   {year}: {event}\")\n\nprint(\"\\n🚀 Ready for live Claude Code enhancement!\")